In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from Bio.KEGG import REST
from Bio import SeqIO
import ast
import time
import DLKcat
import torch
from rdkit import Chem
from collections import defaultdict
import math
import re
import cobra
from tqdm import tqdm 

/home/wuke/anaconda3/envs/kcat/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### DLKcatCurrencyMets.tsv

### DLKcatIgnoreMets.tsv

In [2]:
miss_met_id =[
's_0803', 's_0804', 's_0805', 's_0806', 's_0807', 's_0808', 's_0809', 's_0810', 's_2808', 's_2976', 's_2994', 's_3226', 's_3449', 's_3657',##h20
's_0793', 's_0794', 's_0795', 's_0796', 's_0797', 's_0798', 's_0799', 's_0800', 's_0801', 's_0802', 's_2783', 's_3094', 's_3146', 's_3164',#H+
's_1275', 's_1276', 's_1277', 's_1278', 's_1279', 's_2817',#oxygen
's_0837', 's_0838', 's_0839', 's_0840',#hydrogen peroxide
's_0419', 's_0420', 's_0421',#ammonium
's_1322', 's_1323', 's_1324', 's_1325', 's_1326', 's_1329', 's_2966', 's_2977', 's_2995', 's_3228', 's_3536', 's_4293',#phosphate
's_0633', 's_0635', 's_0636', 's_0637', 's_0638', 's_2834', 's_2860', 's_3095', 's_4157',#diphosphate
's_1467', 's_1468', 's_4016',#sulphate
's_0841', 's_3906', 's_4263',#hydrogen sulfide
's_4010', 's_4113', 's_4159',#thiosulfate
's_1373', 's_1374', 's_3776',#potassium
's_1437', 's_1438', 's_3775',#sodium
's_3777', 's_3778', 's_3926', 's_4200',#chloride
's_4013', 's_4014', 's_4204',#Mg(2+)
's_3801', 's_3802', 's_4202',#Mn(2+)
's_3822', 's_3823', 's_4026', 's_4027', 's_4203',#Zn(2+)
's_3880', 's_3882', 's_4197', 's_4199',#Ca(2+)
's_4019', 's_4020', 's_4201', #Cu2(+)
's_0924', 's_0925', 's_0926',#iron(2+)
's_3855', 's_3936', 's_4031', #iron(3+)

's_0434', 's_0435', 's_0437', 's_0438', 's_0439', 's_2831', 's_2840', 's_2856', 's_3341', 's_3359', 's_3881', 's_4196',#ATP
's_0394', 's_0395', 's_0397', 's_0398', 's_0399', 's_3324', 's_3342', 's_3360', 's_3883', 's_3987', 's_4198',#ADP
's_0423', 's_0424', 's_0425', 's_0426', 's_2833', 's_2842', 's_2859', 's_4160',#AMP
's_1203', 's_1204', 's_1205', 's_1206', 's_2818', 's_3753',#NADH
's_1198', 's_1199', 's_1200', 's_1201', 's_1202', 's_2820',#NAD
's_1212', 's_1213', 's_1214', 's_1215', 's_2799', 's_2952',#NADPH
's_1207', 's_1208', 's_1210', 's_1211', 's_2800', 's_2953',#NADP(+)
's_0689', 's_0690',#FADH2
's_0687', 's_0688',#FAD
]

In [3]:
#dlkcat
atom_dict_path ='./dlkcat_input/atom_dict.pickle'
bond_dict_path = './dlkcat_input/bond_dict.pickle'
fingerprint_dict_path = './dlkcat_input/fingerprint_dict.pickle'
edge_dict_path = './dlkcat_input/edge_dict.pickle'
word_dict_path = './dlkcat_input/sequence_dict.pickle'
yeast8U_sequence_smiles_pre_path = '../../../Results/kcat_km_predict/yeast8U_sequence_smiles_prepare.csv'


#output 
# yeast8U_kcat_predict_path = '../../Data/kcat_km_predict/yeast8U_kcat_predict.csv'
yeast8U_kcat_predict_path = '../../../Results/kcat_km_predict/yeast8U_kcat_predict.csv'

In [4]:
kcat_pre_pd = pd.read_csv(yeast8U_sequence_smiles_pre_path)
kcat_pre_pd

,rea_id,gene,met_id,reactant_met_id,product_met_id,SMILES,reactant_SMILES,product_SMILES,Sequence
0,r_0001_1,YEL071W,s_0025,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC(C(=O)O)O,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTAAHPVAQLTAEAYPKVKRNPNFKVLDSEDLAYFRSILSNDEILN...
1,r_0001_2,YEL071W,s_0709,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTAAHPVAQLTAEAYPKVKRNPNFKVLDSEDLAYFRSILSNDEILN...
2,r_0001_3,YJR048W,s_0025,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC(C(=O)O)O,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHS...
3,r_0001_4,YJR048W,s_0709,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHS...
4,r_0001_5,YEL039C,s_0025,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC(C(=O)O)O,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MAKESTGFKPGSAKKGATLFKTRCQQCHTIEEGGPNKVGPNLHGIF...
...,...,...,...,...,...,...,...,...,...
214239,rxn1363_138,YNL267W,s_3958,"['s_3958', 'sn_22']","['s_0188', 's_3998']",C(C(C(=O)[O-])O)O,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MHKASSSKKSFDDTIELKKNEQLLKLINSSEFTLHNCVELLCKHSE...
214240,rxn1363_139,YDR051C,sn_22,"['s_3958', 'sn_22']","['s_0188', 's_3998']",O=P(O)(O)OCC1OC(O[C@]2(CO)OC(CO)C(O)C2O)C(O)C(...,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MCEENVHVSEDVAGSHGSFTNARPRLIVLIRHGESESNKNKEVNGY...
214241,rxn1363_140,YDR051C,s_3958,"['s_3958', 'sn_22']","['s_0188', 's_3998']",C(C(C(=O)[O-])O)O,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MCEENVHVSEDVAGSHGSFTNARPRLIVLIRHGESESNKNKEVNGY...
214242,rxn1363_141,YAL038W,sn_22,"['s_3958', 'sn_22']","['s_0188', 's_3998']",O=P(O)(O)OCC1OC(O[C@]2(CO)OC(CO)C(O)C2O)C(O)C(...,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MSRLERLTSLNVVAGSDLRRTSIIGTIGPKTNNPETLVALRKAGLN...


### kcat prediction

In [5]:
kcat_pre_pd['kcat'] = ''

In [6]:
atom_dict = DLKcat.load_pickle(atom_dict_path)
bond_dict = DLKcat.load_pickle(bond_dict_path)
fingerprint_dict = DLKcat.load_pickle(fingerprint_dict_path)
edge_dict = DLKcat.load_pickle(edge_dict_path)
word_dict = DLKcat.load_pickle(word_dict_path)

In [7]:
def split_sequence(sequence, ngram):
    sequence = '-' + sequence + '='
    # print(sequence)
    # words = [word_dict[sequence[i:i+ngram]] for i in range(len(sequence)-ngram+1)]

    words = list()
    for i in range(len(sequence)-ngram+1) :
        try :
            words.append(word_dict[sequence[i:i+ngram]])
        except :
            word_dict[sequence[i:i+ngram]] = 0
            words.append(word_dict[sequence[i:i+ngram]])

    return np.array(words)
    # return word_dict

def create_atoms(mol):
    """Create a list of atom (e.g., hydrogen and oxygen) IDs
    considering the aromaticity."""
    # atom_dict = defaultdict(lambda: len(atom_dict))
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    # print(atoms)
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    # atoms = list()
    # for a in atoms :
    #     try: 
    #         atoms.append(atom_dict[a])
    #     except :
    #         atom_dict[a] = 0
    #         atoms.append(atom_dict[a])

    return np.array(atoms)

def create_ijbonddict(mol):
    """Create a dictionary, which each key is a node ID
    and each value is the tuples of its neighboring node
    and bond (e.g., single and double) IDs."""
    # bond_dict = defaultdict(lambda: len(bond_dict))
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict

def extract_fingerprints(atoms, i_jbond_dict, radius):
    """Extract the r-radius subgraphs (i.e., fingerprints)
    from a molecular graph using Weisfeiler-Lehman algorithm."""

    # fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    # edge_dict = defaultdict(lambda: len(edge_dict))

    if (len(atoms) == 1) or (radius == 0):
        fingerprints = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges
            (i.e., r-radius subgraphs or fingerprints)."""
            fingerprints = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                # fingerprints.append(fingerprint_dict[fingerprint])
                # fingerprints.append(fingerprint_dict.get(fingerprint))
                try :
                    fingerprints.append(fingerprint_dict[fingerprint])
                except :
                    fingerprint_dict[fingerprint] = 0
                    fingerprints.append(fingerprint_dict[fingerprint])

            nodes = fingerprints

            """Also update each edge ID considering two nodes
            on its both sides."""
            _i_jedge_dict = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    # edge = edge_dict[(both_side, edge)]
                    # edge = edge_dict.get((both_side, edge))
                    try :
                        edge = edge_dict[(both_side, edge)]
                    except :
                        edge_dict[(both_side, edge)] = 0
                        edge = edge_dict[(both_side, edge)]

                    _i_jedge_dict[i].append((j, edge))
            i_jedge_dict = _i_jedge_dict

    return np.array(fingerprints)

def create_adjacency(mol):
    adjacency = Chem.GetAdjacencyMatrix(mol)
    return np.array(adjacency)

def dump_dictionary(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dict(dictionary), file)

def load_tensor(file_name, dtype):
    return [dtype(d).to(device) for d in np.load(file_name + '.npy', allow_pickle=True)]

class Predictor(object):
    def __init__(self, model):
        self.model = model

    def predict(self, data):
        predicted_value = self.model.forward(data)

        return predicted_value

# load pretrained model
# fingerprint_dict = DLKcat.load_pickle('../Script/dlkcat_input/fingerprint_dict.pickle')
# edge_dict = DLKcat.load_pickle('../Script/dlkcat_input/edge_dict.pickle')
# word_dict = DLKcat.load_pickle('../Script/dlkcat_input/sequence_dict.pickle')

# def DLkcat(fingerprint_dict,word_dict,edge_dict,Sequence,smiles):

n_fingerprint = len(fingerprint_dict)
n_word = len(word_dict)
n_edge = len(edge_dict)
radius=2
ngram=3
dim=10
# dim=5
layer_gnn=3
side=5
window=11
layer_cnn=3
layer_output=3
lr=1e-3
lr_decay=0.5
decay_interval=10
weight_decay=1e-6
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# torch.manual_seed(1234)
Kcat_model = DLKcat.KcatPrediction(device, n_fingerprint, n_word, 2*dim, layer_gnn, window, layer_cnn, layer_output).to(device)
Kcat_model.load_state_dict(torch.load('./dlkcat_input/all--radius2--ngram3--dim20--layer_gnn3--window11--layer_cnn3--layer_output3--lr1e-3--lr_decay0.5--decay_interval10--weight_decay1e-6--iteration50', map_location=device))
# print(state_dict.keys())
# model.eval()
predictor = Predictor(Kcat_model)

def DLkcat(Sequence,smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    atoms = create_atoms(mol)
    i_jbond_dict = create_ijbonddict(mol)
    fingerprints = extract_fingerprints(atoms, i_jbond_dict, radius)
    adjacency = create_adjacency(mol)
    #print(adjacency)
    #print(fingerprints)
    #print(i_jbond_dict)
    # Kcat_values = list()
    try:
        sequence = Sequence
        
        words = split_sequence(sequence,ngram)
                        # print(words)
                        # proteins.append(words)
        #print(words)
        fingerprints = torch.LongTensor(fingerprints).to(device)
        #print(fingerprints)
        adjacency = torch.FloatTensor(adjacency).to(device)
        words = torch.LongTensor(words).to(device)

        inputs = [fingerprints, adjacency, words]
        #print(inputs)

        prediction = predictor.predict(inputs)
        Kcat_log_value = prediction.item()
        #print(Kcat_log_value)
        Kcat_value = '%.4f' %math.pow(2,Kcat_log_value)
        
    except:
        Kcat_value = '#'
    
    return(Kcat_value)
    # Kcat_values.append(Kcat_value)

In [8]:
kcat_pre_pd['SMILES_new'] = kcat_pre_pd['SMILES'].apply(lambda x: x.split('.'))
kcat_pre_pd['SMILES_new'] = kcat_pre_pd['SMILES_new'].apply(lambda x: '.'.join([smile for smile in x if smile not in ["[Cl-]", "[Fe+2]", "[Fe]", "[Na+]"]]))
kcat_pre_pd

,rea_id,gene,met_id,reactant_met_id,product_met_id,SMILES,reactant_SMILES,product_SMILES,Sequence,kcat,SMILES_new
0,r_0001_1,YEL071W,s_0025,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC(C(=O)O)O,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTAAHPVAQLTAEAYPKVKRNPNFKVLDSEDLAYFRSILSNDEILN...,,CC(C(=O)O)O
1,r_0001_2,YEL071W,s_0709,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTAAHPVAQLTAEAYPKVKRNPNFKVLDSEDLAYFRSILSNDEILN...,,CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...
2,r_0001_3,YJR048W,s_0025,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC(C(=O)O)O,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHS...,,CC(C(=O)O)O
3,r_0001_4,YJR048W,s_0709,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MTEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHS...,,CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...
4,r_0001_5,YEL039C,s_0025,"['s_0025', 's_0709']","['s_0710', 's_1399']",CC(C(=O)O)O,"['CC(C(=O)O)O', 'CC1=C(C2=CC3=NC(=CC4=C(C(=C([...",['CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C...,MAKESTGFKPGSAKKGATLFKTRCQQCHTIEEGGPNKVGPNLHGIF...,,CC(C(=O)O)O
...,...,...,...,...,...,...,...,...,...,...,...
214239,rxn1363_138,YNL267W,s_3958,"['s_3958', 'sn_22']","['s_0188', 's_3998']",C(C(C(=O)[O-])O)O,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MHKASSSKKSFDDTIELKKNEQLLKLINSSEFTLHNCVELLCKHSE...,,C(C(C(=O)[O-])O)O
214240,rxn1363_139,YDR051C,sn_22,"['s_3958', 'sn_22']","['s_0188', 's_3998']",O=P(O)(O)OCC1OC(O[C@]2(CO)OC(CO)C(O)C2O)C(O)C(...,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MCEENVHVSEDVAGSHGSFTNARPRLIVLIRHGESESNKNKEVNGY...,,O=P(O)(O)OCC1OC(O[C@]2(CO)OC(CO)C(O)C2O)C(O)C(...
214241,rxn1363_140,YDR051C,s_3958,"['s_3958', 'sn_22']","['s_0188', 's_3998']",C(C(C(=O)[O-])O)O,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MCEENVHVSEDVAGSHGSFTNARPRLIVLIRHGESESNKNKEVNGY...,,C(C(C(=O)[O-])O)O
214242,rxn1363_141,YAL038W,sn_22,"['s_3958', 'sn_22']","['s_0188', 's_3998']",O=P(O)(O)OCC1OC(O[C@]2(CO)OC(CO)C(O)C2O)C(O)C(...,"['C(C(C(=O)[O-])O)O', 'O=P(O)(O)OCC1OC(O[C@]2(...","['C(C(C(=O)O)OP(=O)(O)O)O', 'C(C1C(C(C(C(O1)OC...",MSRLERLTSLNVVAGSDLRRTSIIGTIGPKTNNPETLVALRKAGLN...,,O=P(O)(O)OCC1OC(O[C@]2(CO)OC(CO)C(O)C2O)C(O)C(...


In [9]:
for index,row in tqdm(kcat_pre_pd.iterrows(),total=len(kcat_pre_pd)):
    try:
        kcat_pred = DLkcat(row['Sequence'],row['SMILES_new'])
    except: 
        kcat_pred = None
    kcat_pre_pd.loc[index,'kcat'] = kcat_pred

100%|██████████| 214244/214244 [12:07<00:00, 294.40it/s]


In [10]:
print(kcat_pre_pd.shape)
kcat_pre_pd = kcat_pre_pd[kcat_pre_pd['kcat']!=None]
print(kcat_pre_pd.shape)
kcat_pre_pd = kcat_pre_pd[~kcat_pre_pd['kcat'].isna()]
print(kcat_pre_pd.shape)

(214244, 11)
(214244, 11)
(214244, 11)


In [11]:
kcat_pre_pd.to_csv(yeast8U_kcat_predict_path,index=None)